## Grant Planning package with People and Money

### Installation
In addition to jupyter notebook, your machine should have these:
```
pip3 install matplotlib
pip3 install pandas
pip3 install openpyxl
pip3 install xlrd
```

### Usage
Change directory to where the notebooks (.ipynb) files are and 
```
jupyter notebook
```
The code will expect to find an Excel file about projects, how much is 
awarded and spent in various accounting categories. The code will also 
expect an Excel file of transactions, about what has been spent on which grant 
in various categories.

There are two Excel files that come from the People and Money system:
`project.xls` and `transaction.xlsx`
 
There are four supporting concepts to understand. Three of these rely
on additional JSON files. After running the following code snippets, 
you can clear the cell content by selecting the cell then from the menu
'Cell/Current Outputs/Clear'

---
### Run
This code covers an interval of months, from a month in the past
that corresponds to the same month of the `project.xls`, to a month
in the future. Actual expenditure is from the run start until the 
date of the `transactions.xlsx` file.

In [1]:
import util
run = util.run('Aug-22', 'Nov-23')
run.print()

Run is from Aug-22 to Nov-23 (15 months)


---
Directory where the data files can be found

In [2]:
#data_dir = 'toydata'
data_dir = 'toydata2'

---
### People
A JSON file contains last names of the personnel, with 
their [spine points](https://www.ed.ac.uk/human-resources/pay-reward/pay/pay-scales) and salary. There is a multiplier to convert 
salary for the person into cost for the grant. There is a python
module people.py. You can run the cell below to see the results.

In [3]:
import people
people = people.people(data_dir+'/people.json')
people.print()

Monthly cost per full-time person
with cost/salary ratio  1.345

Each line is: Name (spine) Cost
      Bloggs (     49) £ 6929
      Wilson (     40) £ 5315


---
### Grants
Each grant is a combination of a hand-made file with short names and colours, 
together with the projects Excel file from P&M. The two are combined into a 
single file grants.json that is used for the plots and tables.

In [4]:
import grants
g = grants.grants(data_dir+'/mygrants.json')
g.print()

ComfortIV
   Cushions for Comfort phase IV
  P&M project  00000012_00000016

MedievalVII
   Medieval dog activities phase VII
  P&M project  00000022_00000026



This file `mygrants.json` is human editable, just a choice of short names and colours really. 
It is meant to be supplemented with information from the `projects.xls`. 
Sometimes the P&M reported end time for the grant is not appropriate, and it can be 
specificed in this file to override the spreadsheet from P&M.

In [5]:
g.from_projects  (data_dir+'/project_Aug-22.xls', 'Aug-22')

The combined result 
is written to a file `grants.json` that is used henceforth.

In [6]:
g.write(data_dir+'/grants.json')
g.print()

As of beginning of Aug-22
ComfortIV
   Cushions for Comfort phase IV
  P&M project  00000012_00000016
   Start 01-Oct-22, End 29-Jun-23 (9 months)
   Awarded     Spent    Category
    100000      20000   Salary
     10000       2000   Travel
         0          0   Equipment
         0          0   Consumables

MedievalVII
   Medieval dog activities phase VII
  P&M project  00000022_00000026
   Start 01-Jan-22, End 29-Jun-24 (30 months)
   Awarded     Spent    Category
    300000      20000   Salary
     40000       1000   Travel
     10000          0   Equipment
     20000          0   Consumables



Grants are assigned colours in the `mygrants.json` file, 
which can be seen by running the following cell:

In [7]:
from IPython.display import display, HTML
out = g.colour_chart()
display(HTML(out))

---
### Assign
This is the forecasting part of the system, where FTE and salary are 
combined to see the effect on the grant. There is no forecasting of non-salary expenses.

People are assigned to grants through records like:
["Davidson",    "Gaia21To22",    "Jan-22",    0.5],
which says the person Davidson will work 50% on the grant Gaia21To22, 
starting Jan-22. The notebook Assign_Tool.ipynb shows who is working 
on which grant month by month, together with the sum of all the 
FTE percentages, which should add to 1.0 for a full-time employee.
Running the following cell will be helpful in editing the assignFTE.json file.

In [8]:
import assign
af = assign.assign(
    data_dir+'/assignFTE.json', 
    data_dir+'/grants.json', 
    data_dir+'/people.json', 
    run)
af.print()

Bloggs
   Aug-22 (0.50): 0.50 on MedievalVII| 
   Sep-22 (0.50): 0.50 on MedievalVII| 
   Oct-22 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
   Nov-22 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
   Dec-22 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
   Jan-23 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
   Feb-23 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
   Mar-23 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
   Apr-23 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
   May-23 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
   Jun-23 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
   Jul-23 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
   Aug-23 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
   Sep-23 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
   Oct-23 (1.00): 0.50 on  ComfortIV| 0.50 on MedievalVII| 
Wilson
   Aug-22 (1.00): 1.00 on MedievalVII| 
   Sep-22 (1.00): 1.00 on MedievalVII| 
   Oct-22 (1.00): 1.00 on MedievalVII| 
   Nov

---
### Non-salary expenses
Each grant may be awarded money in several expense catagories. 
However some of these are ignored by this system, 
since the PI has no control over their expenditure. Some of the controllable
expense categories are lumped together. This can be seen in the code `util.py`,
where currently we have:

In [9]:
import util
print('All considered "Salary":',      util.category_salary)
print()
print('All considered "Consumables":', util.category_consumables)
print()
print('All considered "Travel":',      util.category_travel)
print()
print('All considered "Equipment":',   util.category_equipment)
print()
print('And these are ignored:',        util.category_ignore)

All considered "Salary": ['Research Investigator', 'Research Assistant']

All considered "Consumables": ['Consumables - Research Other Costs', 'Consumables - IT', 'Consumables - Telephone and Communication', 'Staff Conf Course and Seminar Fees']

All considered "Travel": ['Travel and Subsistence']

All considered "Equipment": ['Equipment']

And these are ignored: ['nan', 'Grant Income', 'Financial Resources', 'Directly Allocated - Co-Principal Investigator Staff', 'Directly Allocated - Estates Costs', 'Directly Allocated - Infrastructure Technician Costs', 'Directly Allocated - Principal Investigator Staff', 'Directly Allocated - Co-Principal Investigator Staff', 'Directly Allocated Research Administration Staff', 'Indirect Costs']


---
### Notebook Grants.ipynb
Each grant is shown in order of the short name, with the amount awarded and spent, 
in each of the categories. Note: we always assume the start of the run is the same as 
the date of the `project.xls` file.

Four plots correcond to the four expense categories, showing actual expense from the
`transactions.xlsx` file, as well as a grey line of constant slope from what is spent at the 
beginning of the run to the assumption of full spend at the end of the grant. 
The Salary plot also shows forecast expense to the end of the run.

The table 'Forecast Salary' is from the `assign.json` file,
together with cost summed ove people, cumulative spend on salary,
and remaining balance. The table 'Salary Spending' is similar, but with actual expenses
from the `transactions.xlsx` file. The table 'Category Spending' shows 
expenditure in the four categories.

### Notebook People.ipynb
Each person is shown with two plots. The left is the forecast of their
FTE on the various grants, and the right is their actual costs
charged to different grants, converted to FTE.